In [1]:
# import config variables
from config import *

# import libraries
import argparse
import multiprocessing as mp
import numpy as np, pandas as pd, math
np.seterr(divide='ignore', invalid='ignore')
import sys, threading
import gzip, h5py, os
import statsmodels.api as sm
import tqdm
import time
import re

import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms

from functools import partial
from sklearn.calibration import calibration_curve, CalibrationDisplay
from sklearn.linear_model import LogisticRegression
import glob


In [139]:
# set up base config
sim = 1
prefix = f"output/sim{sim}/"
variant_type = 'top_diff'

## True PRS

In [140]:
# load true risk data
true_f = h5py.File(f"{prefix}true_prs/prs_m_1000_h2_0.5_{variant_type}.hdf5","r")

# label for each individual in the data
true_labels = true_f["labels"][()]
# true PRS
true_standardized_prs = true_f["G"][()]
# environmental effect from true PRS
true_E = true_f["E"][()]

true_train_cases_ceu = true_f["train_cases_ceu"][()]
true_train_controls_ceu = true_f["train_controls_ceu"][()]
true_train_cases_yri = true_f["train_cases_yri"][()]
true_train_controls_yri = true_f["train_controls_yri"][()]

# group labels
zip_true_prs_and_E = list(zip(true_labels, true_standardized_prs, true_E))

true_f.close()

In [142]:
# make dataframe of true risk scores per each person in the true risk prs data
true_prs_df = pd.DataFrame(zip_true_prs_and_E, columns=["group", "true_prs", "true_E"])
true_prs_df['group'] = true_prs_df['group'].map(lambda x: x.decode("utf-8"))

# subset of european and african ancestry individuals
exclude_admix_mask = true_prs_df['group'].str.contains('ceu|yri') 
true_prs_df = true_prs_df[exclude_admix_mask]

# add column for total trait liability
true_prs_df = true_prs_df.assign(true_L = lambda x: x.true_prs + x.true_E) # where L is the total trait liability

# subset european population
ceu_mask = true_prs_df['group'].str.contains('ceu')  
true_prs_df_ceu = true_prs_df[ceu_mask].copy()

# subset african population
yri_mask = true_prs_df['group'].str.contains('yri')  
true_prs_df_yri = true_prs_df[yri_mask].copy()


In [143]:
# Phenotype (P) is 1 for the top 5% of people ranked by L (total trait liability)

def get_true_phenotype_outcome(group_df):
    # select the tail end of the total trait liability distribution as cases
    select_percentage_of_cases = 0.05

    selected_cases_in_group = group_df.sort_values('true_L').tail(int(group_df.shape[0] * select_percentage_of_cases))
    first_case = selected_cases_in_group.head(1).true_L.squeeze()
    last_case = selected_cases_in_group.tail(1).true_L.squeeze()

    true_outcome = group_df['true_L'].between(first_case, last_case).astype(int)
    
    return true_outcome

true_prs_df_ceu['true_outcome'] = get_true_phenotype_outcome(true_prs_df_ceu)
true_prs_df_yri['true_outcome'] = get_true_phenotype_outcome(true_prs_df_yri)


In [113]:
num_simulations = 10000

def simulate_non_genetic_effect(group_df):
    # e*_i ~ N(0, 1-h^2)
    # mean = 0, variance (standard deviation) = (1-h^2)
    mean, variance = 0, 1-H2
    norm_dist = np.random.normal(mean, variance, size = int(group_df.shape[0]))
    z_score = (norm_dist - norm_dist.mean())/ norm_dist.std()
    
    # E*_i = (e*_i - (mean of e*_i/SD of e*_i)) * sqrt(variance)) <- where variance is 1-h^2
    expected_E = (z_score / norm_dist.std()) * math.sqrt(variance) 

    # standardized non-genetic effect (environmental noise)
    return expected_E


def simulate_phenotype_occurence(group_df, expected_E):
    true_risk_scores = group_df["true_prs"]
    group_L_df = pd.DataFrame(list(zip(true_risk_scores, expected_E)), columns=["true_prs", "expected_E"])

    # total trait liability [L*_i] (expected_L) -> L*_i = G_i + E*_i
    group_L_df = group_L_df.assign(expected_L = lambda x: x.true_prs + x.expected_E)

    # select the tail end of the total trait liability distribution as cases
    select_percentage_of_cases = 0.05
    selected_cases = group_L_df.sort_values('expected_L').tail(int(group_L_df.shape[0] * select_percentage_of_cases))
    first_case = selected_cases.head(1).expected_L.squeeze()
    last_case = selected_cases.tail(1).expected_L.squeeze()

    expected_P = group_L_df['expected_L'].between(first_case, last_case).astype(int).to_numpy()
    
    return expected_P


In [114]:
def expected_E_multi_run_wrapper(group_df, group_name):
    with mp.Pool() as pool:
        print("Expected E for {}".format(group_name))
        expected_E_for_group = list(tqdm.tqdm(pool.imap(simulate_non_genetic_effect, [group_df for i in range(num_simulations)]), total=num_simulations))

    return expected_E_for_group


def simulated_phenotype_multi_run_wrapper(group_df, group_expected_Es, group_name):
    p_func = partial(simulate_phenotype_occurence, group_df)
    
    with mp.Pool() as pool:
        print("Expected P for {}".format(group_name))
        expected_group_P = list(tqdm.tqdm(pool.imap(p_func, [e for e in group_expected_Es]), total=len(group_expected_Es)))

    return expected_group_P


In [115]:
# # expected standardized non-genetic effect per group
# expected_E_CEU = expected_E_multi_run_wrapper(true_prs_df_ceu, "CEU")
# expected_E_YRI = expected_E_multi_run_wrapper(true_prs_df_yri, "YRI")


In [116]:
# # expected occurrence of phenotype per group
# expected_occurrence_P_CEU = simulated_phenotype_multi_run_wrapper(true_prs_df_ceu, expected_E_CEU, "CEU")
# expected_occurrence_P_YRI = simulated_phenotype_multi_run_wrapper(true_prs_df_yri, expected_E_YRI, "YRI")


In [117]:
# def expected_phenotype_probability(expected_Ps, group_df):
#     expected_group_phenotype_probability = [sum(x)/num_simulations for x in zip(*expected_Ps)] 
    
#     return expected_group_phenotype_probability

# true_prs_df_ceu['true_prs_probability_of_outcome'] = expected_phenotype_probability(expected_occurrence_P_CEU, true_prs_df_ceu)
# true_prs_df_yri['true_prs_probability_of_outcome'] = expected_phenotype_probability(expected_occurrence_P_YRI, true_prs_df_yri)


## Empirical PRS

In [144]:
emp_variant_type = variant_type
prefix = 'output/sim1/'

In [147]:
# glob all empirical prs data files
emp_prs_files_dir_prefix = f'{prefix}emp_prs/{emp_variant_type}/'
file_name_glob_pattern = 'emp_prs_m_*_identifier_*[0-9]*.hdf5'

emp_files = glob.glob(emp_prs_files_dir_prefix + file_name_glob_pattern)


In [148]:
def load_all_emp_data(emp_files):
    all_emp_prs_list = []

    for file in emp_files:
        cases = re.search('weights_(.*?)cases_identifier', file).group(1)
        identifier = re.search('identifier_(.*?).hdf5', file).group(1)
        snps = re.search('1_(.*?)_snps_', file).group(1)
        weights = re.search('cases_(.*?)_weights', file).group(1)
    
        # open emp prs file
        f = h5py.File(file, 'r')
        
        # extra individuals and their empirical risk scores
        emp_prs = f['X'][()]
        emp_labels = f["labels"][()]
        
        zip_data = list(zip(emp_labels, emp_prs))
        
        # create a dictionary of the file identifier, number of cases sampled, population snps, population weights,
        # and the labeled individuals and their risk scores
        data_dict = {'id': identifier, 'cases': cases, 'snps': snps, 'weights': weights, 'emp_prs': zip_data}
        
        all_emp_prs_list.append(data_dict)
        
        f.close()
    
    return all_emp_prs_list
    
# load all empirical risk data into list from files
all_emp_prs = load_all_emp_data(emp_files)


In [149]:
def get_emp_prs_cases_controls_data(identifier):
    suffix = f"identifier_{identifier}"
    prefix = 'output/sim1/'
    
    cases_dir_prefix = f'{prefix}emp_prs/{emp_variant_type}/cases/'
    ceu_cases_pattern = f"{cases_dir_prefix}CEU_train_cases"
    yri_cases_pattern = f"{cases_dir_prefix}YRI_train_cases"

    control_dir_prefix = f'{prefix}/emp_prs/{emp_variant_type}/control/'
    ceu_control_pattern = f"{control_dir_prefix}CEU_control"
    yri_control_pattern = f"{control_dir_prefix}YRI_control"

    cases = glob.glob(cases_dir_prefix+'*')
    controls = glob.glob(control_dir_prefix+'*')
    
    # file for all ceu cases for the given identifier
    ceu_cases = [i for i in cases if i.startswith(ceu_cases_pattern) and i.endswith(suffix)]
    # file for all yri cases for the given identifier
    yri_cases = [i for i in cases if i.startswith(yri_cases_pattern) and i.endswith(suffix)]

    # file for all ceu control for the given identifier
    ceu_control = [i for i in controls if i.startswith(ceu_control_pattern) and i.endswith(suffix)]
    # file for all yri cases for the given identifier
    yri_control = [i for i in controls if i.startswith(yri_control_pattern) and i.endswith(suffix)]

    cases_control_files_dict = {
        'ceu_cases': ceu_cases, 
        'yri_cases': yri_cases,
        'ceu_control': ceu_control,
        'yri_control': yri_control
    }
    
    for label, file in cases_control_files_dict.items():
        assert len(file) == 1, "the list confirmed has 1 item"
        with open(file[0]) as temp_file:
            l = [line.rstrip('\n') for line in temp_file]
            # remove first zero element from all
            l.pop(0)
        # dictionary value is now a list of indices from the given case or control file
        cases_control_files_dict[label] = l

    return cases_control_files_dict


## Generating Learning Curve Data for Empirical Risk Score



In [150]:
def generate_learning_curve_data(list_of_emp_prs_files):
    list_of_probabilities_per_emp_prs_file = []
        
    for file in list_of_emp_prs_files:
        cases = file['cases']
        snps = file['snps']
        weights = file['weights']
        identifier = file['id']

        zip_emp_prs = file['emp_prs']
                        
        if np.isnan(zip_emp_prs[1][1]):
            continue
            
        emp_prs_df = pd.DataFrame(zip_emp_prs, columns=["group", "emp_prs"])
        emp_prs_df['group'] = emp_prs_df['group'].map(lambda x: x.decode("utf-8"))

        # load the data for ceu and yri separately
        ceu_mask = emp_prs_df['group'].str.contains('ceu') 
        # each row is a person in european population
        emp_prs_df_ceu = emp_prs_df[ceu_mask].copy()

        yri_mask = emp_prs_df['group'].str.contains('yri')
        # each row is a person in african population
        emp_prs_df_yri = emp_prs_df[yri_mask].copy()
        
        # add true phenotype outcome to empirical risk scores for each group
        emp_prs_df_ceu['true_outcome'] = true_prs_df_ceu['true_outcome'].copy()
        emp_prs_df_yri['true_outcome'] = true_prs_df_yri['true_outcome'].copy()
        
        emp_prs_df_ceu['training_size'] = cases
        emp_prs_df_yri['training_size'] = cases
        
        emp_prs_df_ceu['identifier'] = identifier
        emp_prs_df_yri['identifier'] = identifier
        
        emp_prs_df_ceu['snps'] = snps
        emp_prs_df_yri['snps'] = snps
        
        emp_prs_df_ceu['weights'] = weights
        emp_prs_df_yri['weights'] = weights
        
        # subset cases and control training data for each identifier
        cases_control_files_dict = get_emp_prs_cases_controls_data(identifier)
                
        ceu_training_data = cases_control_files_dict['ceu_cases'] + cases_control_files_dict['ceu_control']
        ceu_training_data = [f'ceu_{i}' for i in ceu_training_data]
        
        yri_training_data = cases_control_files_dict['yri_cases'] + cases_control_files_dict['yri_control']
        yri_training_data = [f'yri_{i}' for i in yri_training_data]

        emp_prs_df_ceu['is_in_training_data'] = np.where(emp_prs_df_ceu.group.isin(ceu_training_data), 1, 0)
        emp_prs_df_yri['is_in_training_data'] = np.where(emp_prs_df_yri.group.isin(yri_training_data), 1, 0)
    
        print(f"Saving {emp_variant_type} data for CEU with {cases} cases and {snps} snps..")
        emp_prs_df_ceu.to_csv(f"learning_curve_data/{emp_variant_type}/ceu_emp_prs_{emp_variant_type}_cases_{cases}_snps_{snps}_id_{identifier}.csv", index=False)

        print(f"Saving {emp_variant_type} data for YRI with {cases} cases and {snps} snps..")
        emp_prs_df_yri.to_csv(f"learning_curve_data/{emp_variant_type}/yri_emp_prs_{emp_variant_type}_cases_{cases}_snps_{snps}_id_{identifier}.csv", index=False)


In [152]:
generate_learning_curve_data(all_emp_prs)


Saving top_diff data for CEU with 3000 cases and yri snps..
Saving top_diff data for YRI with 3000 cases and yri snps..
Saving top_diff data for CEU with 5000 cases and ceu snps..
Saving top_diff data for YRI with 5000 cases and ceu snps..
Saving top_diff data for CEU with 4000 cases and ceu snps..
Saving top_diff data for YRI with 4000 cases and ceu snps..
Saving top_diff data for CEU with 1000 cases and yri snps..
Saving top_diff data for YRI with 1000 cases and yri snps..
Saving top_diff data for CEU with 8000 cases and yri snps..
Saving top_diff data for YRI with 8000 cases and yri snps..
Saving top_diff data for CEU with 10000 cases and ceu snps..
Saving top_diff data for YRI with 10000 cases and ceu snps..
Saving top_diff data for CEU with 6000 cases and yri snps..
Saving top_diff data for YRI with 6000 cases and yri snps..
Saving top_diff data for CEU with 400 cases and yri snps..
Saving top_diff data for YRI with 400 cases and yri snps..
Saving top_diff data for CEU with 10000 